In [108]:
import pandas as pd
import os
import spacy
import pyvi
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Get Data

In [109]:
# Get data
data = pd.DataFrame()
current_dir = os.getcwd()
for dir in ['train', 'test', 'validation']:
    df = pd.read_csv(f"{os.path.join(current_dir, 'data', dir)}.csv")
    print(f'File {dir}.csv có {df.shape[0]} dòng.')
    data = pd.concat([data,df], axis=0)
print(f'Data tổng có {data.shape[0]} dòng.')

File train.csv có 11426 dòng.
File test.csv có 3166 dòng.
File validation.csv có 1583 dòng.
Data tổng có 16175 dòng.


In [110]:
# Preview data
data.sample(10)

,sentence,sentiment,topic
4877,"đề mở , giảng viên nhiệt tình .",2,0
218,thầy cô cần cho thêm bài tập tài liệu về nhà c...,0,0
8761,"rút kinh nghiệm về sau , em sẽ ở nhà code bài ...",0,1
7074,thầy đảm bảo tốt về giờ lên lớp .,1,0
4327,giáo viên ít hướng dẫn bài tập về nhà .,2,0
752,"ngiêm túc , cứng rắn về vấn đề tự học .",2,0
9938,"giáo viên giỏi , nhiệt tình .",2,0
4074,các dụng cụ thực hành không được cung cấp đầy ...,0,2
10081,thầy dạy dễ hiểu và khá gần gũi với sinh viên ...,2,0
3056,giáo viên nên cho nhiều ví dụ nhỏ trên lớp hơn...,0,0


In [111]:
# Convert sentiment and topic
# Convert sentiment and topic column
sentiment = {'0':'negative', '1':'neutral', '2':'positive'}
topic = {'0':'lecturer', '1':'training_program', '2':'facility', '3':'others'}

data['target'] = data['sentiment'].astype(str).map(sentiment)
data['topic_'] = data['topic'].astype(str).map(topic)

data.sample(10)

,sentence,sentiment,topic,target,topic_
609,"giảng viên dạy rất chi tiết , dễ hiểu , nội du...",2,0,positive,lecturer
1801,thầy dạy học phần 2 hơi khó chịu .,0,0,negative,lecturer
1672,phần thực hành này học 3 buổi .,1,1,neutral,training_program
1174,"cô đến lớp đúng giờ , giảng dễ hiểu .",2,0,positive,lecturer
1201,cô rất vui và rất gần gũi với học sinh .,2,0,positive,lecturer
986,"vui tính , thân thiện .",2,0,positive,lecturer
0,giáo trình chưa cụ thể .,0,1,negative,training_program
1622,"giảng viên hướng dẫn rõ , chỉ dẫn từng sinh vi...",2,0,positive,lecturer
8320,"thứ nhất thầy không đảm bảo được giờ lên lớp ,...",0,0,negative,lecturer
4556,bài học liên quan tới thực tế .,2,0,positive,lecturer


# Preprocess

In [112]:
# Convert to lower
data['sentence'] = data['sentence'].str.lower()

In [113]:
# Remove punctuation
data['sentence'] = data['sentence'].str.replace('[^\w\s]','')

In [114]:
# Tokenization
data['tokens'] = data['sentence'].apply(lambda x: pyvi.ViTokenizer.tokenize(x))

In [115]:
# Remove stopwords
with open('vietnamese-stopwords.txt', 'r', encoding='utf-8') as f:
    stopwords = f.read().splitlines()
data['tokens'] = data['tokens'].apply(lambda x: ' '.join([word for word in x.split() if word not in stopwords]))

In [116]:
data

,sentence,sentiment,topic,target,topic_,tokens
0,slide giáo trình đầy đủ .,2,1,positive,training_program,slide giáo_trình đầy_đủ .
1,"nhiệt tình giảng dạy , gần gũi với sinh viên .",2,0,positive,lecturer,"nhiệt_tình giảng_dạy , gần_gũi sinh_viên ."
2,đi học đầy đủ full điểm chuyên cần .,0,1,negative,training_program,đi học đầy_đủ full chuyên_cần .
3,chưa áp dụng công nghệ thông tin và các thiết ...,0,0,negative,lecturer,áp_dụng công_nghệ thông_tin thiết_bị hỗ_trợ gi...
4,"thầy giảng bài hay , có nhiều bài tập ví dụ ng...",2,0,positive,lecturer,"thầy giảng , bài_tập ví_dụ lớp ."
...,...,...,...,...,...,...
1578,hướng dẫn lab mơ hồ .,0,0,negative,lecturer,hướng_dẫn lab mơ_hồ .
1579,thầy cho chúng em những bài tập mang tính thực...,2,0,positive,lecturer,"thầy bài_tập thực_hành , thực_tiễn thực_sự hài..."
1580,thầy không dạy nhiều chủ yếu cho sinh viên tự ...,0,0,negative,lecturer,thầy dạy chủ_yếu sinh_viên tìm_hiểu .
1581,em muốn đổi tên môn học vì tên môn là lập trìn...,0,1,negative,training_program,đổi môn_học môn lập_trình c fraction cplusplus...


# Build Model

In [117]:
X, y = data['tokens'], data['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [118]:
# TF-IDF
vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features=5000)
X_train_vectorizer = vectorizer.fit_transform(X_train)
X_test_vectorizer = vectorizer.transform(X_test)

In [119]:
model = LogisticRegression()
model.fit(X_train_vectorizer, y_train)

LogisticRegression()

In [121]:
y_pred = model.predict(X_test_vectorizer)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

cfm = confusion_matrix(y_test, y_pred)
print(cfm)

report = classification_report(y_test, y_pred)
print(report)

Accuracy: 0.8168143416443437
[[1961    2  309]
 [ 126   21   70]
 [ 376    6 1982]]
              precision    recall  f1-score   support

           0       0.80      0.86      0.83      2272
           1       0.72      0.10      0.17       217
           2       0.84      0.84      0.84      2364

    accuracy                           0.82      4853
   macro avg       0.79      0.60      0.61      4853
weighted avg       0.81      0.82      0.80      4853



# Save Model

In [ ]:
import joblib

In [ ]:
current_dir = os.getcwd()
save_dir = os.path.join(os.path.dirname(current_dir), 'model')
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

joblib.dump(model, os.path.join(save_dir, 'model.pkl'))
joblib.dump(vectorizer, os.path.join(save_dir, 'vectorizer.pkl'))

['/content/model/vectorizer.pkl']